In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sklearn.datasets as ds
from typing import List
import matplotlib.pyplot as plt
from utils import *
from solvers import EPOSolver, LinearScalarizationSolver
from models import *
import json
from pathlib import Path
import pygmo as pg

In [3]:
data = pd.read_csv('/home/aalcantara/HNvsPearce/data/kin8nm.csv', sep=',')
scaler = StandardScaler()
data = pd.DataFrame(scaler.fit_transform(data), index = data.index, columns = data.columns)
y = data[['y']]
X = data.drop('y', axis=1)

In [3]:
'''data = ds.fetch_california_housing(return_X_y=False, as_frame=True)
dataf = data.frame
scaler = StandardScaler()
dataf = pd.DataFrame(scaler.fit_transform(dataf), index = dataf.index, columns = dataf.columns)
y = dataf[['MedHouseVal']]
X = dataf.drop('MedHouseVal', axis=1)'''

"data = ds.fetch_california_housing(return_X_y=False, as_frame=True)\ndataf = data.frame\nscaler = StandardScaler()\ndataf = pd.DataFrame(scaler.fit_transform(dataf), index = dataf.index, columns = dataf.columns)\ny = dataf[['MedHouseVal']]\nX = dataf.drop('MedHouseVal', axis=1)"

In [3]:
torch.cuda.empty_cache()
seed = 1234
set_seed(seed)
set_logger()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Usando device = ', device)

hnet = POPI_hyper(input_target=81, target_hidden_dim=4, target_hidden_size=50).to(device)
net = POPI_target(input_target=81, target_hidden_dim=4, target_hidden_size=50).to(device)

# ---------
# Task loss
# ---------

loss1 = AIW_loss()
loss2 = PICP_alpha_loss()


lr = 0.0005
wd = 0.
optimizer = torch.optim.Adam(hnet.parameters(), lr=lr, weight_decay=wd)

# ------
# solver
# ------

solvers = dict(ls=LinearScalarizationSolver, epo=EPOSolver)

solver_type = 'ls'
solver_method = solvers[solver_type]
if solver_type == 'epo':
    solver = solver_method(n_tasks=2, n_params=count_parameters(hnet))
else:
    # ls
    solver = solver_method(n_tasks=2)

# ------
# data
# ------

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

train_features = torch.Tensor(X_train.values)
train_targets = torch.Tensor(y_train.values)
test_features = torch.Tensor(X_test.values)
test_targets = torch.Tensor(y_test.values)

train_ds = TensorDataset(train_features, train_targets)
test_ds = TensorDataset(test_features, test_targets)

bs = 2500
train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=len(test_ds), shuffle=True)

n_rays = 200
min_angle = 1e-3
max_angle = np.pi / 2 - 1e-3
test_rays = circle_points(n_rays, min_angle=min_angle, max_angle=max_angle)


Usando device =  cuda:0


In [4]:
# 0.9759, 4/100
# boston 0.9268, 4/100
# yacht 0.9882, 4/100
# super 0.9420, 4/150
val_results = dict()
test_results = dict()
out_dir="outputs_Concrete"
no_val_eval = False
full_test = True
eval_every = 50
alpha=0.25
best_hv = -1
last_eval = -1
for epoch in range(0,5000):
    ###### ENTRENAMIENTO ######
    
    for x_tr, y_tr in train_loader:
        hnet.train()
        optimizer.zero_grad()
        x_tr = x_tr.to(device)
        y_tr  = y_tr.to(device)
        
        if alpha > 0:
            ray = np.random.dirichlet((alpha, alpha), 1)
            ray = torch.from_numpy(ray.astype(np.float32).flatten()).to(device)
        else:
            alpha = torch.empty(1, ).uniform_(0., 1.)
            ray = torch.tensor([alpha.item(), 1 - alpha.item()]).to(device)
        
        weights = hnet(ray)
        low, up = net(x_tr, weights)
        
        p_int = torch.cat((low, up), axis=1)
        l1 = loss1(p_int, y_tr[:, [0]])
        l2 = loss2(p_int, y_tr[:, [0]])
        losses = torch.stack((l1, l2))
        
        ray = ray.squeeze(0)
        loss = solver(losses, ray, list(hnet.parameters()))
        
        loss.backward()
        optimizer.step()
    
    
    ###### VALIDACIÓN ######
    if (epoch + 1) >= 1000:
        if (epoch + 1) % eval_every == 0:
            last_eval = epoch
            if not no_val_eval:
                epoch_results, hv = evaluate(hypernet=hnet, targetnet=net, loader=test_loader, rays=test_rays, device=device)
                val_results[f'epoch_{epoch + 1}'] = epoch_results
                if hv > best_hv:
                    torch.save(hnet.state_dict(), 'best_model_hn_main.pt')
                    best_hv = hv
                    best_dict = epoch_results
                    best_epoch = (epoch + 1)
                print('Epoch: ', epoch+1, 'Best hv:', best_hv, 'in epoch', best_epoch)
    




2022-06-17 12:29:45,270 - numexpr.utils - INFO - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-06-17 12:29:45,271 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


Epoch:  1000 Best hv: 0.928097366433129 in epoch 1000
Epoch:  1050 Best hv: 0.928097366433129 in epoch 1000
Epoch:  1100 Best hv: 0.9306172399386334 in epoch 1100
Epoch:  1150 Best hv: 0.9306172399386334 in epoch 1100
Epoch:  1200 Best hv: 0.9306172399386334 in epoch 1100
Epoch:  1250 Best hv: 0.9306172399386334 in epoch 1100
Epoch:  1300 Best hv: 0.9306172399386334 in epoch 1100
Epoch:  1350 Best hv: 0.9306172399386334 in epoch 1100
Epoch:  1400 Best hv: 0.9306172399386334 in epoch 1100
Epoch:  1450 Best hv: 0.9307289960840126 in epoch 1450
Epoch:  1500 Best hv: 0.9327327955381856 in epoch 1500


KeyboardInterrupt: 

In [5]:
out_dir = Path(out_dir)
out_dir.mkdir(parents=True, exist_ok=True)

with open(Path(out_dir) / "val_results.json", "w") as file:
    json.dump(val_results, file)

In [6]:
PINPS = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
hnet.load_state_dict(torch.load('best_model_hn_main.pt'))
for pinp in PINPS:
    test_epoch_results = test(hypernet=hnet, targetnet=net, loader=test_loader, val_dict=best_dict, PINP=pinp, rays=test_rays,device=device)
    test_results[f'PredInt_{pinp*100}'] = test_epoch_results
with open(Path(out_dir) / "test_results.json", "w") as file:
    json.dump(test_results, file)